# Использование доступных моделей
Данный ноутбук показывает базовый порядок работы с библиотекой.
- загрузка датасета
- деление на трейн и тест
- обучение модели
- сравнение с бейзлайном

Документацию можно собрать из папки `docs` командой `make html`. Она создаст папку `_build` с документацией -- `sponge-bob-magic/docs/_build/html/index.html`

In [24]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [25]:
import os, sys

parent_dir = os.path.split(os.getcwd())[0]
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

Целевая архитектура библиотеки - ЛД 3.0, поэтому используем Spark.
Для отладки моделей (не на кластере) создаём локальную сессию.

Объект `State` позволяет использовать одну и ту же сессию Spark в разных объектах.
Если сессия уже инициализированна, то её нужно передать при инициализации `State`. Модули библиотеки, которым необходимо использовать спарк-сессию, например для конвертации из пандас в спарк, будут искать сессию именно в `State`.

По умолчанию создастся дефолтная сессия. Простой способ получить сессию с заданным количеством выделенной памяти -- функция `get_spark_session` модуля `session_handler`.

In [26]:
from sponge_bob_magic.session_handler import State

spark = State().session
spark

## Подготовка данных <a name='data-preparator'></a>
Библиотека содержит утилиты для скачивания и парсинга популярных датасетов для рекомендаций.
Если датасет не найден в заданной директории, он будет автоматически скачан и обработан.

Данные датасета доступны в виде `pandas.DataFrame` атрибутов объекта.
Посмотреть доступные данные можно с помощью метода `info`.

In [27]:
from sponge_bob_magic.datasets import MovieLens

data = MovieLens("1m")
data.info()

ratings


,user_id,item_id,relevance,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968



users


,user_id,gender,age,occupation,zip_code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117



items


,item_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance


Внутренний формат данных в либе -- спарк датафрейм. Сплиттеры умеют втоматически конвертировать пандас датафреймы, но модели сейчас ожидают именно спарк, причем с заданными обязательными колонками: `user_id, item_id, timestamp, relevance, context`. 

Получить данные в нужном формате можно с помощью `DataPreparator.transform_log`, который создаст нужные колонки, если их нет.

In [28]:
from sponge_bob_magic.data_preparator import DataPreparator

log = DataPreparator().transform(
    data=data.ratings,
    columns_names={
        "user_id": "user_id",
        "item_id": "item_id",
        "relevance": "relevance",
        "timestamp": "timestamp"
    }
)

Библиотека содержит различные схемы валидации рекомендательных систем, встречающиеся в литературе.

`UserSplitter` отбирает для теста некоторое количество или долю объектов для каждого пользователя.

In [29]:
from sponge_bob_magic.splitters import UserSplitter

splitter = UserSplitter(
    drop_cold_items=True,
    drop_cold_users=True,
    item_test_size=1,
    user_test_size=500,
    seed=1234,
    shuffle=True
)
train, test = splitter.split(log)
(
    train.count(), 
    test.count()
)

(999709, 500)

## NMF
Простейший пример использования DL в рекомендациях

In [30]:
from sponge_bob_magic.models import NeuroMFRec

nmf = NeuroMFRec(
    learning_rate=0.01,
    epochs=10,
    embedding_dimension=100
)

In [31]:
%%time

nmf.fit(log=train)

CPU times: user 3min 5s, sys: 5.65 s, total: 3min 11s
Wall time: 4min 6s


In [32]:
%%time

recs = nmf.predict(
    k=10,
    log=train,
    filter_seen_items=True
)

CPU times: user 16.9 s, sys: 313 ms, total: 17.2 s
Wall time: 39.9 s


В библиотеке реализованы различные метрики качества рекомендательных систем, встречающихся в литературе.
Их можно использовать напрямую, либо запоминать результаты с помощью класса `Experiment`.

In [33]:
from sponge_bob_magic.metrics import HitRate, NDCG
from sponge_bob_magic.experiment import Experiment

metrics = Experiment(test, {NDCG(): 10, 
                            HitRate(): 10})

metrics.add_result("NMF", recs)
metrics.df.loc["NMF"]

18-Mar-20 16:15:21, root, WARNING: Значение метрики может быть неожиданным:пользователи в recommendations и ground_truth различаются!


18-Mar-20 16:16:13, root, WARNING: Значение метрики может быть неожиданным:пользователи в recommendations и ground_truth различаются!


HitRate@10    0.001987
NDCG@10       0.000956
Name: NMF, dtype: float64

## ALS
Библиотека также содержит классические алгоритмы рекомендаций, например, матричную факторизацию

In [34]:
from sponge_bob_magic.models import ALSRec

als = ALSRec(rank=100)

In [35]:
%%time
als.fit(log=train)

CPU times: user 49.1 ms, sys: 7.07 ms, total: 56.2 ms
Wall time: 41.7 s


In [36]:
%%time
recs = als.predict(
    k=10,
    users=test.select('user_id').distinct(),
    items=test.select('item_id').distinct(),
    context='no_context',
    log=train,
    filter_seen_items=True
)

CPU times: user 27.3 ms, sys: 6.32 ms, total: 33.6 ms
Wall time: 8.16 s


In [37]:
metrics.add_result("ALS", recs)
metrics.df.loc["ALS"]

HitRate@10    0.140000
NDCG@10       0.057582
Name: ALS, dtype: float64

## Своя модель
Для построения своей модели нужно использовать тот же самый split, что и для бейзлайнов

In [38]:
train.toPandas().to_csv("train.csv", index=False)

/Users/shminke-ba/projects/sponge-bob-magic/venv/lib/python3.7/site-packages/pyarrow/__init__.py:157: UserWarning: pyarrow.open_stream is deprecated, please use pyarrow.ipc.open_stream
  warnings.warn("pyarrow.open_stream is deprecated, please use "


In [39]:
!head train.csv

user_id,item_id,relevance,timestamp
1090,1961,4.0,2000-11-23 01:20:08
1090,2428,2.0,2000-11-23 01:27:11
1090,1208,3.0,2000-11-23 01:21:17
1090,296,5.0,2000-11-23 01:06:15
1090,318,5.0,2000-11-23 01:11:51
1090,1704,4.0,2000-11-23 01:12:48
1090,2433,3.0,2000-11-23 01:26:01
1090,1569,4.0,2000-11-23 01:02:22
1090,110,3.0,2000-11-23 01:12:48


Здесь магия: можно взять train и обучить на нём свою любимую модель вне библиотеки

Также нужно выдать рекомендации обученной моделью

In [40]:
recs.toPandas().to_csv("recs.csv", index=False)

Теперь нужно прочитать рекомендации в формате, поддерживаемом библиотекой

In [41]:
recs = DataPreparator().transform(
    path="recs.csv",
    columns_names={
        "user_id": "user_id",
        "item_id": "item_id",
        "relevance": "relevance"
    },
    header=True,
    format_type="csv"
)

и сравнить качество своей модели с бейзлайнамии

In [42]:
metrics.add_result("my_model", recs)
metrics.df

,HitRate@10,NDCG@10
NMF,0.001987,0.000956
ALS,0.140000,0.057582
my_model,0.140000,0.057582
